In [1]:
import numpy as np
import cv2
import os
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Initialize MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2)  
mpDraw = mp.solutions.drawing_utils

# Set the path to your dataset
path_to_data = "./Alphabet_2_hands_signs/"

# Initialize a dictionary to hold file paths for each class
Sign_file_names_dict = {}

# Populate the dictionary with image file paths
for img_dir in os.scandir(path_to_data):
    if img_dir.is_dir():
        sign_name = img_dir.name
        Sign_file_names_dict[sign_name] = []
        for entry in os.scandir(img_dir.path):
            if entry.is_file() and entry.name.lower().endswith(('.png', '.jpg', '.jpeg')):
                Sign_file_names_dict[sign_name].append(entry.path)

# Define the class labels
class_dict = {name: idx for idx, name in enumerate(Sign_file_names_dict.keys())}




In [2]:
print(class_dict)

{'A': 0, 'B': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'J': 7, 'K': 8, 'M': 9, 'N': 10, 'P': 11, 'Q': 12, 'R': 13, 'S': 14, 'T': 15, 'W': 16, 'X': 17, 'Y': 18, 'Z': 19}


In [3]:
Sign_file_names_dict

{'A': ['./Alphabet_2_hands_signs/A\\0.jpg',
  './Alphabet_2_hands_signs/A\\1.jpg',
  './Alphabet_2_hands_signs/A\\10.jpg',
  './Alphabet_2_hands_signs/A\\100.jpg',
  './Alphabet_2_hands_signs/A\\11.jpg',
  './Alphabet_2_hands_signs/A\\12.jpg',
  './Alphabet_2_hands_signs/A\\13.jpg',
  './Alphabet_2_hands_signs/A\\14.jpg',
  './Alphabet_2_hands_signs/A\\15.jpg',
  './Alphabet_2_hands_signs/A\\16.jpg',
  './Alphabet_2_hands_signs/A\\17.jpg',
  './Alphabet_2_hands_signs/A\\18.jpg',
  './Alphabet_2_hands_signs/A\\19.jpg',
  './Alphabet_2_hands_signs/A\\2.jpg',
  './Alphabet_2_hands_signs/A\\20.jpg',
  './Alphabet_2_hands_signs/A\\21.jpg',
  './Alphabet_2_hands_signs/A\\22.jpg',
  './Alphabet_2_hands_signs/A\\23.jpg',
  './Alphabet_2_hands_signs/A\\24.jpg',
  './Alphabet_2_hands_signs/A\\25.jpg',
  './Alphabet_2_hands_signs/A\\26.jpg',
  './Alphabet_2_hands_signs/A\\27.jpg',
  './Alphabet_2_hands_signs/A\\28.jpg',
  './Alphabet_2_hands_signs/A\\29.jpg',
  './Alphabet_2_hands_signs/A\\3.jpg'

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils



# Initialize MediaPipe Hands with max_num_hands set to 2
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,  # Detect up to two hands
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

def load_and_preprocess_images(Sign_file_names_dict, class_dict, image_size=(100, 100)):
    X, y = [], []
    for sign_name, image_paths in Sign_file_names_dict.items():
        for image_path in image_paths:
            # Load the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Error loading image: {image_path}")
                continue
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)

            landmarks = []
            if results.multi_hand_landmarks:
                # Flatten landmarks for all detected hands
                for hand_landmarks in results.multi_hand_landmarks:
                    for lm in hand_landmarks.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])
            # If fewer than two hands are detected, append zeros
            while len(landmarks) < 126:
                landmarks.extend([0.0, 0.0, 0.0])

            # Resize the image
            img_resized = cv2.resize(img, image_size)
            # Normalize the image
            img_normalized = img_resized / 255.0

            # Add the landmarks to X and the class to y
            X.append(landmarks)
            y.append(class_dict[sign_name])

    X = np.array(X)  # This should work now
    y = np.array(y)
    return X, y




X, y = load_and_preprocess_images(Sign_file_names_dict, class_dict)

print(f"Processed {len(X)} images.")
print(f"Class labels: {class_dict}")

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


   #  keras.layers.Dropout(0.5),
   #  keras.layers.Dense(30, activation='relu'),
   #  keras.layers.Dropout(0.5),
   #  keras.layers.Dense(15, activation='relu'),
   #  keras.layers.Dropout(0.5),

    # model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    # model.add(layers.MaxPooling2D((2, 2)))
    # model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    # model.add(layers.MaxPooling2D((2, 2)))
    # model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model = models.Sequential([
    layers.InputLayer(input_shape=(126,)), 
    layers.Dropout(0.3),
    layers.Dense(126, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(np.unique(y_encoded)), activation='softmax')  # Output layer
])



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=2, validation_data=(X_test, y_test))


In [12]:
print(f"Shape of X: {X.shape}")
print(f"Number of elements in X[0]: {len(X[0])}")

Shape of X: (2016, 32, 32, 3)
Number of elements in X[0]: 32


In [13]:
print(X[0])

[[[0.36470588 0.45490196 0.64313725]
  [0.38823529 0.4627451  0.65882353]
  [0.42352941 0.47058824 0.6627451 ]
  ...
  [0.11764706 0.05882353 0.05098039]
  [0.10980392 0.05098039 0.04313725]
  [0.11372549 0.05490196 0.04705882]]

 [[0.3372549  0.43529412 0.62745098]
  [0.35294118 0.43921569 0.64313725]
  [0.38823529 0.45098039 0.6627451 ]
  ...
  [0.10588235 0.04705882 0.04313725]
  [0.10196078 0.04705882 0.03921569]
  [0.10588235 0.05490196 0.04313725]]

 [[0.29803922 0.38823529 0.58039216]
  [0.30196078 0.38823529 0.58823529]
  [0.3372549  0.41568627 0.62745098]
  ...
  [0.10196078 0.04313725 0.03921569]
  [0.10196078 0.04313725 0.03529412]
  [0.10196078 0.05098039 0.03921569]]

 ...

 [[0.05882353 0.01176471 0.00392157]
  [0.05882353 0.01176471 0.00392157]
  [0.0627451  0.01568627 0.00784314]
  ...
  [0.30196078 0.4        0.55686275]
  [0.27843137 0.38039216 0.52941176]
  [0.27843137 0.36862745 0.51764706]]

 [[0.05882353 0.01176471 0.01176471]
  [0.05882353 0.01176471 0.01176471]


In [ ]:
import pickle 

with open('test.pkl', 'wb') as file:
    pickle.dump(model, file)

